1) Problem statement.

"Trips & Travel.Com" company wants to enable and establish a viable business model to expand the customer base.

 One of the ways to expand the customer base is to introduce a new offering of packages. 

Currently, there are 5 types of packages the company is offering * Basic, Standard, Deluxe, Super Deluxe, King.

Looking at the data of the last year, we observed that 18% of the customers purchased the packages. 

However, the marketing cost was quite high because customers were contacted at random without looking at the available information.

The company is now planning to launch a new product i.e. Wellness Tourism Package.

Wellness Tourism is defined as Travel that allows the traveler to maintain, enhance or kick-start a healthy lifestyle,

and support or increase one's

sense of well-being. However, this time company wants to harness the available data of existingand potential customers to make the marketing expenditure 

more efficient.


2) Data Collection.

The Dataset is collected from https://www.kaggle.com/datasets/susant4learning/holiday-package-purchase-prediction

The data consists of 20 column and 4888 rows.

# 004

In [58]:
##importing important libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [59]:
df=pd.read_csv("Travel.csv")
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


# data cleaning

## handling missing values

1.Handling Missing values

2.Handling Duplicates

3.check data type

4.understand the dataset

In [60]:
df.isnull().sum()

CustomerID                    0
ProdTaken                     0
Age                         226
TypeofContact                25
CityTier                      0
DurationOfPitch             251
Occupation                    0
Gender                        0
NumberOfPersonVisiting        0
NumberOfFollowups            45
ProductPitched                0
PreferredPropertyStar        26
MaritalStatus                 0
NumberOfTrips               140
Passport                      0
PitchSatisfactionScore        0
OwnCar                        0
NumberOfChildrenVisiting     66
Designation                   0
MonthlyIncome               233
dtype: int64

In [61]:
df['Gender'].value_counts()

Gender
Male       2916
Female     1817
Fe Male     155
Name: count, dtype: int64

In [62]:
df['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Divorced      950
Single        916
Unmarried     682
Name: count, dtype: int64

In [63]:
df['TypeofContact'].value_counts()

TypeofContact
Self Enquiry       3444
Company Invited    1419
Name: count, dtype: int64

In [64]:
df['Gender']=df["Gender"].replace('Fe Male','Female')
df['MaritalStatus']=df['MaritalStatus'].replace('Single','Unmarried')

In [65]:
df['Gender'].value_counts()

Gender
Male      2916
Female    1972
Name: count, dtype: int64

In [66]:
df['MaritalStatus'].value_counts()

MaritalStatus
Married      2340
Unmarried    1598
Divorced      950
Name: count, dtype: int64

In [67]:
##checaking missing values
##these are the features with nan value
features_with_nan=[features for features in df.columns if df[features].isnull().sum()>=1]
for feature in features_with_nan:
    print(feature,np.round(df[feature].isnull().mean()*100,5),'% missing values')

Age 4.62357 % missing values
TypeofContact 0.51146 % missing values
DurationOfPitch 5.13502 % missing values
NumberOfFollowups 0.92062 % missing values
PreferredPropertyStar 0.53191 % missing values
NumberOfTrips 2.86416 % missing values
NumberOfChildrenVisiting 1.35025 % missing values
MonthlyIncome 4.76678 % missing values


In [68]:
#statistics on numerical columns(null cols)
df[features_with_nan].select_dtypes(exclude='object').describe()

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


# imputing null values
### 1.impute meadian value for age column
### 2.impute mode for type of contract
### 3.impute median for Duraction of pitch
### 4.impute mode for NumberOfFollowup as it is Discrete feature
### 5.impute mode PreferredPropertyStar
### 6.impute median for NumberofTrips
### 7.impute mode for NUmberofChildrenVisiting
### 8.impute median for monthlyincome

In [69]:
#age
df.Age.fillna(df.Age.median(),inplace=True)
#typeofcontract
df.TypeofContact.fillna(df.TypeofContact.mode()[0],inplace=True)
#DuractionOfPitch
df.DurationOfPitch.fillna(df.DurationOfPitch.median(),inplace=True)
#NumberOfFollowups
df.NumberOfFollowups.fillna(df.NumberOfFollowups.mode()[0],inplace=True)
# PreferredProperStar
df.PreferredPropertyStar.fillna(df.PreferredPropertyStar.mode()[0],inplace=True)
#Numberoftrips
df.NumberOfTrips.fillna(df.NumberOfTrips.median(),inplace=True)
#numberof childrenvisiting
df.NumberOfChildrenVisiting.fillna(df.NumberOfChildrenVisiting.mode()[0],inplace=True)
# montlyincome
df.MonthlyIncome.fillna(df.MonthlyIncome.median(),inplace=True)

In [70]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [71]:
df.isnull().sum()

CustomerID                  0
ProdTaken                   0
Age                         0
TypeofContact               0
CityTier                    0
DurationOfPitch             0
Occupation                  0
Gender                      0
NumberOfPersonVisiting      0
NumberOfFollowups           0
ProductPitched              0
PreferredPropertyStar       0
MaritalStatus               0
NumberOfTrips               0
Passport                    0
PitchSatisfactionScore      0
OwnCar                      0
NumberOfChildrenVisiting    0
Designation                 0
MonthlyIncome               0
dtype: int64

df.drop('CustomerID',inplace=True,axis=1)

# Feature Engineering
## Feature Extraction

In [72]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Unmarried,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [73]:
#crete a new column for feature
df['totalvisiting']=df['NumberOfChildrenVisiting']+df['NumberOfPersonVisiting']
df.drop(columns=['NumberOfPersonVisiting','NumberOfChildrenVisiting'],axis=1,inplace=True)

In [74]:
# get all numneric features
num_features=[feature for feature in df.columns if df[feature].dtype!='O']
print("num of numeric fetures",len(num_features))

num of numeric fetures 13


In [75]:
cat_features=[feature for feature in df.columns if df[feature].dtype=='O']
print("num of categorical fetures",len(cat_features))

num of categorical fetures 6


In [76]:
discrete_features=[feature for feature in num_features if len(df[feature].unique())<=25]
print("num of discrete fetures",len(discrete_features))

num of discrete fetures 9


In [77]:
continuous_features=[feature for feature in num_features if feature not in discrete_features]
print("num of continuous fetures",len(continuous_features))

num of continuous fetures 4


In [78]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,totalvisiting
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


# train test split and mosel training

In [79]:
from sklearn.model_selection import train_test_split
x=df.drop(['ProdTaken'],axis=1)
y=df['ProdTaken']

In [80]:
x.head()

,CustomerID,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,totalvisiting
0,200000,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Unmarried,1.0,1,2,1,Manager,20993.0,3.0
1,200001,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,200002,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Unmarried,7.0,1,3,0,Executive,17090.0,3.0
3,200003,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,200004,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [81]:
y.value_counts()

ProdTaken
0    3968
1     920
Name: count, dtype: int64

In [82]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((3910, 18), (978, 18))

In [83]:
y_train.shape,y_test.shape

((3910,), (978,))

In [84]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   CustomerID              4888 non-null   int64  
 1   Age                     4888 non-null   float64
 2   TypeofContact           4888 non-null   object 
 3   CityTier                4888 non-null   int64  
 4   DurationOfPitch         4888 non-null   float64
 5   Occupation              4888 non-null   object 
 6   Gender                  4888 non-null   object 
 7   NumberOfFollowups       4888 non-null   float64
 8   ProductPitched          4888 non-null   object 
 9   PreferredPropertyStar   4888 non-null   float64
 10  MaritalStatus           4888 non-null   object 
 11  NumberOfTrips           4888 non-null   float64
 12  Passport                4888 non-null   int64  
 13  PitchSatisfactionScore  4888 non-null   int64  
 14  OwnCar                  4888 non-null   

In [85]:
cat_featuress=x.select_dtypes(include='object').columns
num_featuress=x.select_dtypes(include=['int64','float64']).columns

In [86]:
cat_featuress

Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
       'MaritalStatus', 'Designation'],
      dtype='object')

In [87]:
num_featuress

Index(['CustomerID', 'Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
       'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'totalvisiting'],
      dtype='object')

In [88]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer=StandardScaler()
oh_transformer=OneHotEncoder(drop='first')
preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_featuress),
        ("StandardScaler",numeric_transformer,num_featuress)
    ]
)

In [89]:
df['ProductPitched'].value_counts()

ProductPitched
Basic           1842
Deluxe          1732
Standard         742
Super Deluxe     342
King             230
Name: count, dtype: int64

In [90]:
preprocessor

ColumnTransformer(transformers=[('OneHotEncoder', OneHotEncoder(drop='first'),
                                 Index(['TypeofContact', 'Occupation', 'Gender', 'ProductPitched',
       'MaritalStatus', 'Designation'],
      dtype='object')),
                                ('StandardScaler', StandardScaler(),
                                 Index(['CustomerID', 'Age', 'CityTier', 'DurationOfPitch', 'NumberOfFollowups',
       'PreferredPropertyStar', 'NumberOfTrips', 'Passport',
       'PitchSatisfactionScore', 'OwnCar', 'MonthlyIncome', 'totalvisiting'],
      dtype='object'))])

In [91]:
##applying Transform in training(fit_transform)
x_train=preprocessor.fit_transform(x_train)
pd.DataFrame(x_train)

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,0.679690,0.782966,-0.382245,-0.774151
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.721400,0.690023,0.282777,-0.725271,1.511598,-0.632399,0.679690,0.782966,-0.459799,0.643615
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-1.020350,0.282777,1.771041,0.418708,-0.632399,0.679690,0.782966,-0.245196,-0.065268
3,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,...,-0.721400,-1.020350,1.284279,-0.725271,-0.127737,-0.632399,1.408395,-1.277194,0.213475,-0.065268
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,2.400396,-1.720227,-0.725271,1.511598,-0.632399,-0.049015,-1.277194,-0.024889,2.061382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3905,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.721400,-0.653841,1.284279,-0.725271,-0.674182,-0.632399,-1.506426,0.782966,-0.536973,0.643615
3906,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.455047,-0.898180,-0.718725,1.771041,-1.220627,-0.632399,1.408395,0.782966,1.529609,-0.065268
3907,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.455047,1.545210,0.282777,-0.725271,2.058043,-0.632399,-0.777720,0.782966,-0.360576,0.643615
3908,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1.455047,1.789549,1.284279,-0.725271,-0.127737,-0.632399,-1.506426,0.782966,-0.252799,0.643615


In [92]:
##apply transformation on test(transform)
x_test=preprocessor.transform(x_test)

In [93]:
x_test

array([[ 0.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.73751038, -0.77415132],
       [ 1.        ,  0.        ,  0.        , ..., -1.2771941 ,
        -0.6704111 , -0.06526803],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.4208322 , -0.77415132],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.78296635,
         0.69001249,  0.64361526],
       [ 1.        ,  0.        ,  0.        , ...,  0.78296635,
        -0.22827818, -0.77415132],
       [ 1.        ,  1.        ,  0.        , ...,  0.78296635,
        -0.44611323,  2.06138184]], shape=(978, 27))

## xgboost classification implemtation

In [94]:
!pip install scikit-learn

In [95]:
!pip install xgboost

In [97]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,precision_score,recall_score,f1_score,roc_auc_score

In [98]:
models={
    "Random Forest":RandomForestClassifier(),
    "Decision Tree":DecisionTreeClassifier(),
    "Logistic Regression":LogisticRegression(),
    "Gradient Boost":GradientBoostingClassifier(),
    "AdaBoost Classifier":AdaBoostClassifier(),
    "xgboost":XGBClassifier()

}
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)#train model

    #Make Prediction
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    #training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred)
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_rocauc_score=roc_auc_score(y_train,y_train_pred)

    #test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred)
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_rocauc_score=roc_auc_score(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("model performance for training set")
    print("accuracy:{:.4F}".format(model_train_accuracy))
    print("f1 score:{:.4f}".format(model_train_f1))
    print("precision:{:.4F}".format(model_train_precision))
    print("recall:{:.4f}".format(model_train_recall))
    print("rocauc_score:{:.4F}".format(model_train_rocauc_score))

    print('----------------------------------------------------')
    print("model performance for test set")
    print("accuracy:{:.4F}".format(model_test_accuracy))
    print("f1 score:{:.4f}".format(model_test_f1))
    print("precision:{:.4F}".format(model_test_precision))
    print("recall:{:.4f}".format(model_test_recall))
    print("rocauc_score:{:.4F}".format(model_test_rocauc_score))
    print('------- ---------   ------------- ----------- ----------')


Random Forest
model performance for training set
accuracy:1.0000
f1 score:1.0000
precision:1.0000
recall:1.0000
rocauc_score:1.0000
----------------------------------------------------
model performance for test set
accuracy:0.9162
f1 score:0.7320
precision:0.9739
recall:0.5864
rocauc_score:0.7913
------- ---------   ------------- ----------- ----------
Decision Tree
model performance for training set
accuracy:1.0000
f1 score:1.0000
precision:1.0000
recall:1.0000
rocauc_score:1.0000
----------------------------------------------------
model performance for test set
accuracy:0.8957
f1 score:0.7182
precision:0.7602
recall:0.6806
rocauc_score:0.8143
------- ---------   ------------- ----------- ----------
Logistic Regression
model performance for training set
accuracy:0.8468
f1 score:0.4279
precision:0.7044
recall:0.3073
rocauc_score:0.6389
----------------------------------------------------
model performance for test set
accuracy:0.8354
f1 score:0.4145
precision:0.6786
recall:0.2984
roc

In [103]:
##hyperparameter training
rf_params={'max_depth':[5,8,15,None,10],
           'max_features':[5,7,'auto',8],
           'min_samples_split':[2,8,15,20],
           'n_estimators':[100,200,500,1000]}
xg_param={
    "n_estimators":[100,200,300],
    'max_depth':[5,8,12,20,30],
    "learning_rate":[0.1,0.01],
    "colsample_bytree":[0.5,0.8,1,0.3,0.4]
                     
}

In [102]:
rf_params

{'max_depth': [5, 8, 15, None, 10],
 'max_features': [5, 7, 'auto', 8],
 'min_samples_split': [2, 8, 15, 20],
 'n_estimators': [100, 200, 500, 1000]}

In [104]:
xg_param

{'n_estimators': [100, 200, 300],
 'max_depth': [5, 8, 12, 20, 30],
 'learning_rate': [0.1, 0.01],
 'colsample_bytree': [0.5, 0.8, 1, 0.3, 0.4]}

In [105]:
# models list for hyperparameter tuning
randomcv_models=[
    ("RF",RandomForestClassifier(),rf_params),
    ("xg",XGBClassifier(),xg_param)
]

In [106]:
randomcv_models

[('RF',
  RandomForestClassifier(),
  {'max_depth': [5, 8, 15, None, 10],
   'max_features': [5, 7, 'auto', 8],
   'min_samples_split': [2, 8, 15, 20],
   'n_estimators': [100, 200, 500, 1000]}),
 ('xg',
  XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric=None, feature_types=None,
                feature_weights=None, gamma=None, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                learning_rate=None, max_bin=None, max_cat_threshold=None,
                max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
                max_leaves=None, min_child_weight=None, missing=nan,
                monotone_constraints=None, multi_strategy=None, n_estimators=None,
                n_jobs=None, num_parallel_tree=None, ...),
  {'n_estima

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

model_param = {}

for name, model, params in randomcv_models:
    random_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=params,
        n_iter=100,
        cv=3,
        verbose=2,
        n_jobs=-1
    )
    random_search.fit(x_train, y_train)
    model_param[name] = random_search.best_params_

for model_name in model_param:
    print(f"------------Best params for {model_name}-------------")
    print(model_param[model_name])

Fitting 3 folds for each of 100 candidates, totalling 300 fits


In [ ]:
models={
    "Random Forest":RandomForestClassifier(n_estimators=1000,min_samples_split=2,max_features=7,max_depth=None),
    "xgboost":XGBClassifier(n_estimators=200,max_depth=12,learning_rate=0.1,colsample_bytree=1)

}
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)#train model

    #Make Prediction
    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    #training set performance
    model_train_accuracy=accuracy_score(y_train,y_train_pred)
    model_train_f1=f1_score(y_train,y_train_pred)
    model_train_precision=precision_score(y_train,y_train_pred)
    model_train_recall=recall_score(y_train,y_train_pred)
    model_train_rocauc_score=roc_auc_score(y_train,y_train_pred)

    #test set performance
    model_test_accuracy=accuracy_score(y_test,y_test_pred)
    model_test_f1=f1_score(y_test,y_test_pred)
    model_test_precision=precision_score(y_test,y_test_pred)
    model_test_recall=recall_score(y_test,y_test_pred)
    model_test_rocauc_score=roc_auc_score(y_test,y_test_pred)

    print(list(models.keys())[i])

    print("model performance for training set")
    print("accuracy:{:.4F}".format(model_train_accuracy))
    print("f1 score:{:.4f}".format(model_train_f1))
    print("precision:{:.4F}".format(model_train_precision))
    print("recall:{:.4f}".format(model_train_recall))
    print("rocauc_score:{:.4F}".format(model_train_rocauc_score))

    print('----------------------------------------------------')
    print("model performance for test set")
    print("accuracy:{:.4F}".format(model_test_accuracy))
    print("f1 score:{:.4f}".format(model_test_f1))
    print("precision:{:.4F}".format(model_test_precision))
    print("recall:{:.4f}".format(model_test_recall))
    print("rocauc_score:{:.4F}".format(model_test_rocauc_score))
    print('----------------------------------------------------')



Random Forest
model performance for training set
accuracy:1.0000
f1 score:1.0000
precision:1.0000
recall:1.0000
rocauc_score:1.0000
----------------------------------------------------
model performance for test set
accuracy:0.9213
f1 score:0.7540
precision:0.9672
recall:0.6178
rocauc_score:0.8064
----------------------------------------------------
Adaboost
model performance for training set
accuracy:0.8537
f1 score:0.4042
precision:0.8398
recall:0.2661
rocauc_score:0.6272
----------------------------------------------------
model performance for test set
accuracy:0.8436
f1 score:0.4000
precision:0.7969
recall:0.2670
rocauc_score:0.6252
----------------------------------------------------


In [ ]:
y_test

144     0
79      0
2098    0
4738    0
2858    1
       ..
2908    0
2059    1
4872    1
843     0
2572    0
Name: ProdTaken, Length: 978, dtype: int64

In [ ]:
# plot roc auc curve
from sklearn.metrics import roc_auc_score,roc_curve
plt.figure()
auc_models=[
    {
        'label':'xgbClassifier',
        'model':XGBClassifier(n_estimators=200,max_depth=12,learning_rate=0.1,colsample_bytree=1),
        'auc':0.8319
    },]

##create loop through all model
for algo in auc_models:
    model=algo['model']
    model.fit(x_train,y_train)
    fpr,tpr,thresholds=roc_curve(y_test,model.predict_proba(x_test)[:,1])
    plt.plot(fpr,tpr,label='%s roc(area=%0.2f)'%(algo['label'],algo['model'],algo['auc']))
    plt.plot([0,1],[0,1],'r--')
    plt.xlim([0.0,1.0])
    plt.ylim([0.0,1.05])
    plt.xlabel('1.specificity(False Positive Rate)')
    plt.ylabel('sensitivity operating characterstic')
    plt.legend(loc='lower right')
    plt.savefig('auc.png')
    plt.show()

TypeError: must be real number, not AdaBoostClassifier

<Figure size 640x480 with 0 Axes>